In [1]:
# cell has tag: "parameters"
start_date = "2000-01-01"
end_date = "2020-01-01"

In [2]:
params = {
    "start_date": start_date, 
    "end_date": end_date
}

In [3]:
%%capture
%pip install bokeh

In [4]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [5]:
%%bigquery sf_stations --params $params
SELECT
    stations.station_id,
    stations.name,
    stations.lat,
    stations.lon,
    COUNT(*) as count
FROM
    `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips` as trips
JOIN 
    `bigquery-public-data.san_francisco_bikeshare.bikeshare_station_info` as stations 
ON 
    trips.start_station_id = stations.station_id
WHERE
    stations.lat BETWEEN 37.7 AND 38.5 AND 
    stations.lon BETWEEN -122.5 AND -122.35 AND
    trips.start_date BETWEEN TIMESTAMP( @start_date ) AND TIMESTAMP( @end_date )
GROUP BY
    stations.station_id, stations.name, stations.lat, stations.lon
ORDER By
    count DESC

In [6]:
sf_stations.head(10)

,station_id,name,lat,lon,count
0,70,Central Ave at Fell St,37.773311,-122.444293,80370
1,50,2nd St at Townsend St,37.780526,-122.390288,56570
2,67,San Francisco Caltrain Station 2 (Townsend St...,37.776639,-122.395526,50283
3,60,8th St at Ringold St,37.774520,-122.409449,46286
4,61,Howard St at 8th St,37.776513,-122.411306,45166
5,74,Laguna St at Hayes St,37.776435,-122.426244,43779
6,55,Webster St at Grove St,37.777053,-122.429558,41647
7,77,11th St at Natoma St,37.773507,-122.416040,39243
8,76,McCoppin St at Valencia St,37.771662,-122.422423,33035
9,64,5th St at Brannan St,37.776754,-122.399018,30195


In [7]:
from bokeh.io import output_file, output_notebook, show
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, LogColorMapper, BasicTicker, ColorBar,
    DataRange1d, PanTool, WheelZoomTool, BoxSelectTool, HoverTool
)
from bokeh.plotting import gmap
from bokeh.models.mappers import ColorMapper, LogColorMapper
from bokeh.palettes import Viridis6

api_key=!curl -s http://metadata/computeMetadata/v1/instance/attributes/api_key -H "Metadata-Flavor: Google"
api_key=api_key[0]

color_mapper = LogColorMapper(palette=Viridis6)
map_options = GMapOptions(lat=37.775, lng=-122.42, map_type="roadmap", zoom=13)
plot = gmap(api_key, map_options=map_options, title="Station Map")
source = ColumnDataSource(sf_stations)

plot.circle(x="lon", y="lat", size=10,
            fill_color={'field': 'count', 'transform': color_mapper},
            fill_alpha=0.8, source=source)

hover = HoverTool(
    tooltips = [
        ("name", "@name"),
        ("loc", "(@lat, @lon)"),
        ("count", "@count")
    ],
    mode='mouse'
)
plot.add_tools(hover)

output_notebook(hide_banner=True)
show(plot)